In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn
from my_utils import Workout_dataset, class_weight_dict
from my_model import make_CNN_RNN_model

import os

In [2]:
# seed 고정
os.environ['PYTHONHASHSEED'] = str(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

tf.random.set_seed(42)
np.random.seed(42)
rn.seed(42)

In [3]:
def scheduler(epoch, lr):
    if (epoch>20) and (lr > 0.00001):
        lr = lr*0.9
        return lr
    else:
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

train_dir = './data/train'
label_dir = './data/data_y_train.csv'
test_dir = './data/test'
test_label_dir = './data/data_y_test.csv'
checkpoint_filepath = "./save/cnn_gru_best.hdf5"

BATCH_SIZE = 64

train_loader = Workout_dataset(
    train_dir, label_dir, mode='Train',
    fold=0, batch_size=BATCH_SIZE, augment=True, shuffle=True)

valid_loader = Workout_dataset(
    train_dir, label_dir, mode='Valid',
    fold=0, batch_size=16, shuffle=True)

test_loader = Workout_dataset(
    test_dir, test_label_dir, mode='Test',
    batch_size=625, shuffle=False)


In [4]:
model = make_CNN_RNN_model(
    lr = 0.001,
    leakyrelu_alpha = 0.2,
    input_kernels = 10,
    input_kernel_width = 3,
    res_kernels = 60,
    res_kernel_width = 3,
    res_regularize_coeff=0.1, #0.2
    res_num = 5, #3
    )

model.summary()

__________________________________________________________________________________
leaky_re_lu_5 (LeakyReLU)       (None, 300, 20)      0           gru_2[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_13 (LeakyReLU)      (None, 300, 20)      0           gru_6[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_17 (LeakyReLU)      (None, 300, 20)      0           gru_8[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 300, 20)      0           gru_4[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 300, 20)      80          leaky_re_lu_1[0][0]              
__________________________

In [5]:

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',min_delta=0.0001,
    patience=20,verbose=1)

history = model.fit_generator(
    generator=train_loader,
    validation_data=valid_loader,
    epochs=2000,
    callbacks=[save_best,early_stop,lr_scheduler],
    class_weight=class_weight_dict)


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/2000
34/34 [==============================] - 31s 264ms/step - loss: 189.7364 - accuracy: 0.3529 - val_loss: 127.2216 - val_accuracy: 0.4453

Epoch 00001: val_loss improved from inf to 127.22160, saving model to ./save/cnn_gru_best.hdf5
Epoch 2/2000
34/34 [==============================] - 6s 167ms/step - loss: 95.1904 - accuracy: 0.5312 - val_loss: 65.4722 - val_accuracy: 0.4531

Epoch 00002: val_loss improved from 127.22160 to 65.47216, saving model to ./save/cnn_gru_best.hdf5
Epoch 3/2000
34/34 [==============================] - 6s 167ms/step - loss: 50.410

In [6]:
model.load_weights(checkpoint_filepath)
model.evaluate_generator(generator=test_loader,verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
1/1 [==============================] - 1s 1s/step - loss: 0.7386 - accuracy: 0.8368


[0.7385903000831604, 0.8367999792098999]